<a href="https://colab.research.google.com/github/ShubhamD-Genzeon/AI_ML_Training_2024/blob/main/Custom_entity_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import spacy

In [49]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [50]:
nlp = spacy.load('en_core_web_lg')
nlp

In [51]:
import json
with open('Corona2.json','r') as f:
  data = json.load(f)

In [69]:
data['examples'][5]

{'id': 'b3481879-a9cd-48db-8526-8ee11c9a72bc',
 'content': "Hantaviruses, usually found in rodents and shrews, were discovered in two species of bats. The Mouyassué virus (MOUV) was isolated from banana pipistrelle bats captured near Mouyassué village in Cote d'Ivoire, West Africa. The Magboi virus was isolated from hairy slit-faced bats found near the Magboi River in Sierra Leone in 2011. They are single-stranded, negative sense, RNA viruses in the Bunyaviridae family.[29][30][31][32]",
 'metadata': {},
 'annotations': [{'id': '13d9af0d-aeb0-4f0c-8a25-0ac0b458f975',
   'tag_id': '2340864c-0876-4ad1-8a34-600324a516d1',
   'end': 12,
   'start': 0,
   'example_id': 'b3481879-a9cd-48db-8526-8ee11c9a72bc',
   'tag_name': 'Pathogen',
   'value': 'Hantaviruses',
   'correct': None,
   'human_annotations': [{'timestamp': '2020-03-21T00:01:26.709000Z',
     'annotator_id': 1,
     'tagged_token_id': '13d9af0d-aeb0-4f0c-8a25-0ac0b458f975',
     'name': 'Ashpat123',
     'reason': 'exploration'

In [53]:
data['examples'][0].keys()

dict_keys(['id', 'content', 'metadata', 'annotations', 'classifications'])

In [54]:
data['examples'][0]['content']

"While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]"

In [55]:
data['examples'][0]['annotations'][0]

{'id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed',
 'tag_id': 'c06bd022-6ded-44a5-8d90-f17685bb85a1',
 'end': 371,
 'start': 360,
 'example_id': '18c2f619-f102-452f-ab81-d26f7e283ffe',
 'tag_name': 'Medicine',
 'value': 'Diosmectite',
 'correct': None,
 'human_annotations': [{'timestamp': '2020-03-21T00:24:32.098000Z',
   'annotator_id': 1,
   'tagged_token_id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed',
   'name': 'Ashpat123',
   'reason': 'exploration'}],
 'model_annotations': []}

In [56]:
training_data= []
for example in data['examples']:
  temp_dict = {}
  temp_dict['text'] = example['content']
  temp_dict['entities'] = []
  for annotation in example['annotations']:
    start = annotation['start']
    end = annotation['end']
    label = annotation['tag_name'].upper()
    temp_dict['entities'].append((start,end,label))
  training_data.append(temp_dict)

print(training_data[0])

{'text': "While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]", 'entities': [(360, 371, 'MEDICINE'), (383, 408, 'MEDICINE'), (104, 112, 'MEDICALCONDITION'), (679,

In [67]:
training_data[5]['text']

"Hantaviruses, usually found in rodents and shrews, were discovered in two species of bats. The Mouyassué virus (MOUV) was isolated from banana pipistrelle bats captured near Mouyassué village in Cote d'Ivoire, West Africa. The Magboi virus was isolated from hairy slit-faced bats found near the Magboi River in Sierra Leone in 2011. They are single-stranded, negative sense, RNA viruses in the Bunyaviridae family.[29][30][31][32]"

In [58]:
training_data[15]['entities']

[(88, 98, 'MEDICINE'), (335, 346, 'MEDICINE'), (52, 63, 'MEDICINE')]

In [59]:
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en")
doc_bin = DocBin()

In [60]:
from spacy.util import filter_spans

for training_example in tqdm(training_data):
  text = training_example['text']
  labels = training_example['entities']
  doc = nlp.make_doc(text)
  ents = []
  for start, end, label in labels:
    span = doc.char_span(start, end, label=label, alignment_mode='contract')
    if span is None:
      print('Skipping entity')
    else:
      print(span)
      ents.append(span)
  filtered_ents = filter_spans(ents)
  doc.ents = filtered_ents
  doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

 39%|███▊      | 12/31 [00:00<00:00, 108.84it/s]

Diosmectite
aluminomagnesium silicate
diarrhea
kaopectate
bismuth compounds
Pepto-Bismol
diarrhea
chemotherapy
constipation
loperamide
diarrhea
Skipping entity
loperamide
Skipping entity
diarrhea
diarrhea
Racecadotril
diarrhea
loss of skin color
Diarrhea
watery bowel movements
dehydration
dehydration
diarrhoea
decrease in responsiveness
fast heart rate
Antiretroviral therapy
ART
ART
HIV
HIV
ART
HIV
] A combined approach with methotrexate and biologics improves ACR50, HAQ scores and RA remission rates.[94] Triple therapy consisting of methotrexate, sulfasalazine and hydroxychloroquine may also effectively control disease activity.[95] Adverse effects should be monitored regularly with toxicity including gastrointestinal, hematologic, pulmonary, and hepatic.[93]
DMARDs
Hydroxychloroquine
cyclosporin
Skipping entity
] A combined approach with methotrexate and biologics improves ACR50, HAQ scores and RA remission rates.[94] Triple therapy consisting of methotrexate, sulfasalazine and hydro

100%|██████████| 31/31 [00:00<00:00, 128.52it/s]

laninamivir
influenza
oseltamivir
sore throat
Influenza
asthma
bacterial pneumonia
vomiting
headache
asthma
coughing
sinus infections
heart
joint pain
stomach flu
diarrhea
feeling
diarrhea
influenza


In [61]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [62]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    153.29    0.55    0.91    0.39    0.01
  7     200        499.25   3640.10   70.00   82.80   60.63    0.70
 14     400        192.16   1153.36   87.20   97.10   79.13    0.87
 22     600         72.20    471.57   93.66   97.45   90.16    0.94
 30     800        117.10    435.09   95.67   95.67   95.67    0.96
 40    1000         97.26    314.46   96.43   97.20   95.67    0.96

Aborted.


In [63]:
nlp_ner = spacy.load("model-best")


In [65]:
doc = nlp_ner("While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.")

colors = {"PATHOGEN": "#F67DE3", "MEDICINE": "#7DF6D9", "MEDICALCONDITION":"#a6e22d"}
options = {"colors": colors}

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [66]:
doc = nlp_ner('''
The patient was prescribed Aspirin for their heart condition.
The doctor recommended Ibuprofen to alleviate the patient's headache.
The patient is suffering from diabetes, and they need to take Metformin regularly.
After the surgery, the patient experienced some post-operative complications, including infection.
The patient is currently on a regimen of Lisinopril to manage their high blood pressure.
The antibiotic course for treating the bacterial infection should be completed as prescribed.
The patient's insulin dosage needs to be adjusted to better control their blood sugar levels.
The physician suspects that the patient may have pneumonia and has ordered a chest X-ray.
The patient's cholesterol levels are high, and they have been advised to take Atorvastatin.
The allergy to penicillin was noted in the patient's medical history.
''')

# colors = {"PATHOGEN": "#F67DE3", "MEDICINE": "#7DF6D9", "MEDICALCONDITION":"#a6e22d"}
# options = {"colors": colors}

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

Hantaviruses, usually found in rodents and shrews, were discovered in two species of bats. The Mouyassué virus (MOUV) was isolated from banana pipistrelle bats captured near Mouyassué village in Cote d'Ivoire, West Africa. The Magboi virus was isolated from hairy slit-faced bats found near the Magboi River in Sierra Leone in 2011. They are single-stranded, negative sense, RNA viruses in the Bunyaviridae family.[29][30][31][32]

In [68]:
doc = nlp_ner('''
Hantaviruses, usually found in rodents and shrews, were discovered in two species of bats. The Mouyassué virus (MOUV) was isolated from banana pipistrelle bats captured near Mouyassué village in Cote d'Ivoire, West Africa. The Magboi virus was isolated from hairy slit-faced bats found near the Magboi River in Sierra Leone in 2011. They are single-stranded, negative sense, RNA viruses in the Bunyaviridae family.[29][30][31][32]
''')

# colors = {"PATHOGEN": "#F67DE3", "MEDICINE": "#7DF6D9", "MEDICALCONDITION":"#a6e22d"}
# options = {"colors": colors}

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)